In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import spacy

# Load the dataset
df = pd.read_csv("mycsv.csv")
df.head(5)

,bullying_words,type_bully
0,Get fucking real dude.,bully
1,She is as dirty as they come and that crook ...,bully
2,why did you fuck it up. I could do it all day...,bully
3,Dude they dont finish enclosing the fucking s...,bully
4,WTF are you talking about Men? No men thats n...,bully


In [90]:
df.sample(10)

,bullying_words,type_bully
1624,"if gov is going to make it ""uncomfortable"" to ...",bully
3649,omg! what are you in the mood for.. i crock po...,bully
11465,theres places to eat at LA Live tho. I need to...,non-bully
11329,Fuck me I just spent bully5 minutes trying to...,non-bully
13461,OMG fucking awesome!,non-bully
9614,;) I'm more of a Doc Ock girl myself. And yeah...,non-bully
15299,tell me 5 facts about yourself,non-bully
5806,i hear ya. iPods can get annoying with that da...,bully
19959,Why do you press harder on the remote-control...,non-bully
10180,he is cuz Syler killed him he knows how to ki...,non-bully


In [91]:
# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Define the preprocess function to handle NaN values
def preprocess(bullying_words):
    if pd.notnull(bullying_words):  
        doc = nlp(bullying_words)
        filtered_tokens = []
        for token in doc:
            if not token.is_stop and not token.is_punct and not token.like_num:
                filtered_tokens.append(token.lemma_)
        return ' '.join(filtered_tokens)
    else:
        return ''  

# Apply the preprocess function to create a new column 'processed_text'
df['processed_text'] = df['bullying_words'].apply(preprocess)


In [92]:
df.head(4)

,bullying_words,type_bully,processed_text
0,Get fucking real dude.,bully,fucking real dude
1,She is as dirty as they come and that crook ...,bully,dirty come crook Rengel Dems fucking cor...
2,why did you fuck it up. I could do it all day...,bully,fuck day let hour ping later sched write book
3,Dude they dont finish enclosing the fucking s...,bully,dude not finish enclose fucking shower hate ...


In [74]:
# Define the input and output columns
X = df['processed_text']
y = df['type_bully']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
# Build the pipeline with TF-IDF Vectorizer and Logistic Regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

In [76]:
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

C:\Users\lejone\.conda\envs\project\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LogisticRegression())])

In [77]:
# Predict on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9927854469160527


C:\Users\lejone\.conda\envs\project\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lejone\.conda\envs\project\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
               precision    recall  f1-score   support

       bully       0.74      0.72      0.73      1745
  nnon-bully       0.00      0.00      0.00         1
   non-bully       0.81      0.59      0.68      2543
       type_       1.00      1.00      1.00    205426

    accuracy                           0.99    209715
   macro avg       0.64      0.58      0.60    209715
weighted avg       0.99      0.99      0.99    209715



C:\Users\lejone\.conda\envs\project\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)*100

99.27854469160528

In [82]:
from joblib import dump, load
# Save the model to a file
dump(pipeline, 'Lejone99.joblib')

['Lejone99.joblib']